In [32]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import lightning.pytorch as pl
import torchmetrics
from torchmetrics import F1Score
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import EarlyStopping

## Load data

### Modified pre-processing per the panspace paper

- Removed flattening of FCGRs
- Normalized FCGRS to [0,1]

In [14]:
# --- Load data ---
mlst_train_df = pd.read_csv('../assets/mlst_train_set.csv')
mlst_test_df = pd.read_csv('../assets/mlst_test_set.csv')
mlst_val_df = pd.read_csv('../assets/mlst_val_set.csv')

serotype_train_df = pd.read_csv('../assets/serotype_train_set.csv')
serotype_test_df = pd.read_csv('../assets/serotype_test_set.csv')
serotype_val_df = pd.read_csv('../assets/serotype_val_set.csv')

subspecies_train_df = pd.read_csv('../assets/subspecies_train_set.csv')
subspecies_test_df = pd.read_csv('../assets/subspecies_test_set.csv')
subspecies_val_df = pd.read_csv('../assets/subspecies_val_set.csv')

kmc5_arrays = os.path.expanduser('~/PROJECTS/GaTech/FCGR_classifier/salmonella_kmc5_arrays/')
kmc7_arrays = os.path.expanduser('~/PROJECTS/GaTech/FCGR_classifier/salmonella_kmc7_arrays/')

def load_kmer_arrays(df, array_dir, suffix):
    arrays = []
    labels = []
    for idx, row in df.iterrows():
        sample_id = row[0]
        label = row[1]
        array_path = os.path.join(array_dir, f"{sample_id}{suffix}.npy")
        if os.path.exists(array_path):
            # No need to flatten for CNN input
            array = np.load(array_path)#.flatten()

            # Normalize array to [0, 1] as per panpace paper
            array = array / array.max()

            arrays.append(array)
            labels.append(label)
        else:
            print(f"Warning: Array file {array_path} not found.")
    return np.array(arrays), np.array(labels)

# MLST
X_train_mlst_5, y_train_mlst_5 = load_kmer_arrays(mlst_train_df, kmc5_arrays, '_k5_k5')
X_val_mlst_5, y_val_mlst_5 = load_kmer_arrays(mlst_val_df, kmc5_arrays, '_k5_k5')
X_test_mlst_5, y_test_mlst_5 = load_kmer_arrays(mlst_test_df, kmc5_arrays, '_k5_k5')

X_train_mlst_7, y_train_mlst_7 = load_kmer_arrays(mlst_train_df, kmc7_arrays, '_k7_k7')
X_val_mlst_7, y_val_mlst_7 = load_kmer_arrays(mlst_val_df, kmc7_arrays, '_k7_k7')
X_test_mlst_7, y_test_mlst_7 = load_kmer_arrays(mlst_test_df, kmc7_arrays, '_k7_k7')

# Serotype
X_train_sero_5, y_train_sero_5 = load_kmer_arrays(serotype_train_df, kmc5_arrays, '_k5_k5')
X_val_sero_5, y_val_sero_5 = load_kmer_arrays(serotype_val_df, kmc5_arrays, '_k5_k5')
X_test_sero_5, y_test_sero_5 = load_kmer_arrays(serotype_test_df, kmc5_arrays, '_k5_k5')

X_train_sero_7, y_train_sero_7 = load_kmer_arrays(serotype_train_df, kmc7_arrays, '_k7_k7')
X_val_sero_7, y_val_sero_7 = load_kmer_arrays(serotype_val_df, kmc7_arrays, '_k7_k7')
X_test_sero_7, y_test_sero_7 = load_kmer_arrays(serotype_test_df, kmc7_arrays, '_k7_k7')

# Subspecies
X_train_sub_5, y_train_sub_5 = load_kmer_arrays(subspecies_train_df, kmc5_arrays, '_k5_k5')
X_val_sub_5, y_val_sub_5 = load_kmer_arrays(subspecies_val_df, kmc5_arrays, '_k5_k5')
X_test_sub_5, y_test_sub_5 = load_kmer_arrays(subspecies_test_df, kmc5_arrays, '_k5_k5')

X_train_sub_7, y_train_sub_7 = load_kmer_arrays(subspecies_train_df, kmc7_arrays, '_k7_k7')
X_val_sub_7, y_val_sub_7 = load_kmer_arrays(subspecies_val_df, kmc7_arrays, '_k7_k7')
X_test_sub_7, y_test_sub_7 = load_kmer_arrays(subspecies_test_df, kmc7_arrays, '_k7_k7')

/scicomp/scratch/rqu4/tmp/ipykernel_2254263/3979613452.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sample_id = row[0]
/scicomp/scratch/rqu4/tmp/ipykernel_2254263/3979613452.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label = row[1]


In [40]:
# Check to make sure it is normalized
print(X_train_sub_5[0])
print(X_train_sub_5[0].shape)

[[0.11731637 0.11785498 0.16777755 ... 0.32464822 0.50700195 0.86235104]
 [0.1970309  0.         0.13293611 ... 0.33797886 0.63000741 0.53921767]
 [0.44189726 0.21921497 0.14003905 ... 0.39298458 0.18720124 0.56749478]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.133946   0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
(32, 32)


# Hyperparameters

These are the hyperparameters I was able to glean from the panspace paper <https://www.biorxiv.org/content/10.1101/2025.03.19.644115v3> and their code <https://github.com/pg-space/panspace/blob/master/src/panspace/dnn/models/cnn_fcgr_levels.py>

In [ ]:
learning_rate = 0.001 # From paper
latent_dim = 128 # From code
hidden_activation = 'relu' # From code

## Loss function

Panspace used a combination of a Rectified Adam optimizer and Lookahead optimizer called "Ranger" optimizer.

In [2]:
# Optimi has an implementation <https://optimi.benjaminwarner.dev/optimizers/ranger/>
!pip install torch-optimi

from optimi import Ranger

Using pip 25.2 from /scicomp/home-pure/rqu4/.conda/envs/pytorch-final-project/lib/python3.13/site-packages/pip (python 3.13)
  Obtaining dependency information for torch-optimi from https://files.pythonhosted.org/packages/a6/92/4b445290b0d8350273f26576b52b8034e19e993a78563156d4b302bb3d2a/torch_optimi-0.3.2-py3-none-any.whl.metadata


## Panspace architecture implementation

Attempted to install tensorflow for an apples-to-apples comparison but failed due to incompatibility with my GPU and lack of clear documentation on building the environment from panspace. Going with oytorch instead

In [48]:
class CNNFCGR(pl.LightningModule):
    def __init__(self, kmer=5, latent_dim=128, hidden_activation='relu', batch_normalization=True, learning_rate=1e-3, num_classes=10):
        super(CNNFCGR, self).__init__()
        self.save_hyperparameters()

        self.kmer = kmer
        self.latent_dim = latent_dim
        self.batch_normalization = batch_normalization
        self.learning_rate = learning_rate
        self.num_classes = num_classes

        # Activation function
        self.act = nn.ReLU()

        # Input shape: (batch, 1, 2**k, 2**k)
        in_channels = 1
        rows = 2 ** kmer

        # Level 2 convolution
        level2_kernel = 2 ** 2
        level2_stride = 2 ** 2
        filters_level2 = 4 * kmer
        self.conv1 = nn.Conv2d(in_channels, filters_level2, kernel_size=level2_kernel, stride=level2_stride)
        self.bn1 = nn.BatchNorm2d(filters_level2)

        # Level 1 convolution
        level1_kernel = 2 ** 1
        level1_stride = 2 ** 1
        filters_level1 = 4 * (kmer - 1)
        self.conv2 = nn.Conv2d(filters_level2, filters_level1, kernel_size=level1_kernel, stride=level1_stride)
        self.bn2 = nn.BatchNorm2d(filters_level1)

        # Flatten and dense
        with torch.no_grad():
            dummy = torch.zeros(1, 1, rows, rows)
            dummy_out = self.forward_features(dummy)
            flattened_dim = dummy_out.shape[1]

        self.fc = nn.Linear(flattened_dim, num_classes)

        # Macro F1 metrics
        self.train_f1 = torchmetrics.F1Score(task='multiclass', num_classes=num_classes, average='macro')
        self.val_f1   = torchmetrics.F1Score(task='multiclass', num_classes=num_classes, average='macro')

    def forward_features(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.act(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.act(x)

        x = torch.flatten(x, 1)
        return x

    def forward(self, x):
        # conv layers
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.act(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.act(x)

        # flatten
        x = torch.flatten(x, 1)

        # final dense layer: outputs latent_dim features
        logits = self.fc(x)  # treat this as logits for classification

        return logits

    def configure_optimizers(self):
        optimizer = Ranger(self.parameters(), lr=self.learning_rate, weight_decay=1e-4)
        return optimizer
    
    def training_step(self, batch, batch_idx):
        x, y = batch  # x: input, y: one-hot labels
        logits = self(x)  # forward pass

        # If y is one-hot, convert to class indices
        if y.ndim == 2:
            y = torch.argmax(y, dim=1)

        # Compute loss
        loss = F.cross_entropy(logits, y)

        # Compute macro F1
        preds = torch.argmax(logits, dim=1)
        f1 = self.train_f1(preds, y)

        # Log loss and F1
        self.log('train_loss', loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log('train_macro_f1', f1, prog_bar=True, on_step=False, on_epoch=True)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        if y.ndim == 2:
            y = torch.argmax(y, dim=1)

        logits = self(x)
        loss = F.cross_entropy(logits, y)

        preds = torch.argmax(logits, dim=1)
        f1 = self.val_f1(preds, y)

        self.log('val_loss', loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log('val_macro_f1', f1, prog_bar=True, on_step=False, on_epoch=True)

    def test_step(self, batch, batch_idx):
        x, y = batch
        if y.ndim == 2:
            y = torch.argmax(y, dim=1)

        logits = self(x)
        loss = F.cross_entropy(logits, y)

        preds = torch.argmax(logits, dim=1)
        f1 = self.val_f1(preds, y)  # or define a separate test_f1 metric if you want

        self.log('test_loss', loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log('test_macro_f1', f1, prog_bar=True, on_step=False, on_epoch=True)


## Prepare data and labels

In [46]:
# -----------------------------
# Scale classes for integer labels
# -----------------------------
def scale(y):
    """
    Converts string labels to integer indices starting from 0.
    Returns scaled labels and the mapping (class -> index).
    """
    classes = np.sort(np.unique(y))
    class_to_idx = {cls: idx for idx, cls in enumerate(classes)}
    y_scaled = np.array([class_to_idx[label] for label in y], dtype=np.int64)
    return y_scaled, classes

def prepare_data(X_train, X_val, X_test, y_train, y_val, y_test):
    # Convert labels to integer indices
    y_train_scaled, classes = scale(y_train)
    y_val_scaled, _ = scale(y_val)
    y_test_scaled, _ = scale(y_test)

    # Convert inputs to tensors and add channel dimension
    # Assume X_* are already (num_samples, H, W)
    X_train_tensor = torch.tensor(X_train).unsqueeze(1).float()  # (N, 1, H, W)
    X_val_tensor   = torch.tensor(X_val).unsqueeze(1).float()
    X_test_tensor  = torch.tensor(X_test).unsqueeze(1).float()

    y_train_tensor = torch.tensor(y_train_scaled, dtype=torch.long)
    y_val_tensor   = torch.tensor(y_val_scaled, dtype=torch.long)
    y_test_tensor  = torch.tensor(y_test_scaled, dtype=torch.long)

    # Create datasets and dataloaders
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    val_dataset   = TensorDataset(X_val_tensor, y_val_tensor)
    test_dataset  = TensorDataset(X_test_tensor, y_test_tensor)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader   = DataLoader(val_dataset, batch_size=64, shuffle=False)
    test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)

    return train_loader, val_loader, test_loader, classes

## Run model

In [50]:
def run_model(x_train, X_val, X_test, y_train, y_val, y_test, kmer=5):
    # -----------------------------
    # Prepare data
    # -----------------------------
    train_loader, val_loader, test_loader, classes = prepare_data(
        x_train, X_val, X_test,
        y_train, y_val, y_test
    )

    # -----------------------------
    # Instantiate model
    # -----------------------------
    num_classes = len(classes)      # number of unique labels

    model = CNNFCGR(kmer=kmer, latent_dim=128, learning_rate=1e-3, num_classes=num_classes)

    # -----------------------------
    # Train model
    # -----------------------------
    trainer = pl.Trainer(
        max_epochs=50,
        accelerator='auto',
        devices=1,
        enable_progress_bar=True
    )
    trainer.fit(model, train_loader, val_loader)

    # -----------------------------
    # Test model
    # -----------------------------
    results = trainer.test(model, dataloaders=test_loader)

    return results


## Run on kmer=5

In [53]:
results_k5 = []

# -----------------------------
# MLST
# -----------------------------
results = run_model(
    X_train_mlst_5, X_val_mlst_5, X_test_mlst_5,
    y_train_mlst_5, y_val_mlst_5, y_test_mlst_5,
    kmer=5
)
f1_val = results[0].get('val_macro_f1', None)
f1_test = results[0]['test_macro_f1']

results_k5.append({
    "dataset": "MLST",
    "kmer": 5,
    "f1_val_macro": f1_val,
    "f1_test_macro": f1_test
})

# -----------------------------
# Serotype
# -----------------------------
results = run_model(
    X_train_sero_5, X_val_sero_5, X_test_sero_5,
    y_train_sero_5, y_val_sero_5, y_test_sero_5,
    kmer=5
)
f1_val = results[0].get('val_macro_f1', None)
f1_test = results[0]['test_macro_f1']

results_k5.append({
    "dataset": "Serotype",
    "kmer": 5,
    "f1_val_macro": f1_val,
    "f1_test_macro": f1_test
})

# -----------------------------
# Subspecies
# -----------------------------
results = run_model(
    X_train_sub_5, X_val_sub_5, X_test_sub_5,
    y_train_sub_5, y_val_sub_5, y_test_sub_5,
    kmer=5
)
f1_val = results[0].get('val_macro_f1', None)
f1_test = results[0]['test_macro_f1']

results_k5.append({
    "dataset": "Subspecies",
    "kmer": 5,
    "f1_val_macro": f1_val,
    "f1_test_macro": f1_test
})

# -----------------------------
# Save k5 results
# -----------------------------
results_df_k5 = pd.DataFrame(results_k5)
results_df_k5.to_csv("../results/panspace_cnn_macro_f1_results_k5.csv", index=False)
print("K5 complete")


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type              | Params | Mode 
-------------------------------------------------------
0 | act      | ReLU              | 0      | train
1 | conv1    | Conv2d            | 340    | train
2 | bn1      | BatchNorm2d       | 40     | train
3 | conv2    | Conv2d            | 1.3 K  | train
4 | bn2      | BatchNorm2d       | 32     | train
5 | fc       | Linear            | 59.6 K | train
6 | train_f1 | MulticlassF1Score | 0      | train
7 | val_f1   | MulticlassF1Score | 0      | train
-------------------------------------------------------
61.3 K    Trainable params
0         Non-trainable params
61.3 K    Total params
0.245     Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode


Epoch 49: 100%|██████████| 232/232 [00:02<00:00, 112.70it/s, v_num=7, val_loss=0.310, val_macro_f1=0.827, train_loss=0.198, train_macro_f1=0.886]    

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 232/232 [00:02<00:00, 110.79it/s, v_num=7, val_loss=0.310, val_macro_f1=0.827, train_loss=0.198, train_macro_f1=0.886]


...

Epoch 49: 100%|██████████| 6/6 [00:00<00:00, 72.30it/s, v_num=9, val_loss=0.623, val_macro_f1=0.931, train_loss=0.626, train_macro_f1=0.858]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 186.38it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.6471318006515503     │
│       test_macro_f1       │     0.88706374168396      │
└───────────────────────────┴───────────────────────────┘

K5 complete


## Run on kmer=7

In [54]:
results_k7 = []

# -----------------------------
# MLST
# -----------------------------
results = run_model(
    X_train_mlst_7, X_val_mlst_7, X_test_mlst_7,
    y_train_mlst_7, y_val_mlst_7, y_test_mlst_7,
    kmer=7
)
f1_val = results[0].get('val_macro_f1', None)
f1_test = results[0]['test_macro_f1']

results_k7.append({
    "dataset": "MLST",
    "kmer": 7,
    "f1_val_macro": f1_val,
    "f1_test_macro": f1_test
})

# -----------------------------
# Serotype
# -----------------------------
results = run_model(
    X_train_sero_7, X_val_sero_7, X_test_sero_7,
    y_train_sero_7, y_val_sero_7, y_test_sero_7,
    kmer=7
)
f1_val = results[0].get('val_macro_f1', None)
f1_test = results[0]['test_macro_f1']

results_k7.append({
    "dataset": "Serotype",
    "kmer": 7,
    "f1_val_macro": f1_val,
    "f1_test_macro": f1_test
})

# -----------------------------
# Subspecies
# -----------------------------
results = run_model(
    X_train_sub_7, X_val_sub_7, X_test_sub_7,
    y_train_sub_7, y_val_sub_7, y_test_sub_7,
    kmer=7
)
f1_val = results[0].get('val_macro_f1', None)
f1_test = results[0]['test_macro_f1']

results_k7.append({
    "dataset": "Subspecies",
    "kmer": 7,
    "f1_val_macro": f1_val,
    "f1_test_macro": f1_test
})

# -----------------------------
# Save k7 results
# -----------------------------
results_df_k7 = pd.DataFrame(results_k7)
results_df_k7.to_csv("../results/panspace_cnn_macro_f1_results_k7.csv", index=False)
print("K7 complete")


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type              | Params | Mode 
-------------------------------------------------------
0 | act      | ReLU              | 0      | train
1 | conv1    | Conv2d            | 476    | train
2 | bn1      | BatchNorm2d       | 56     | train
3 | conv2    | Conv2d            | 2.7 K  | train
4 | bn2      | BatchNorm2d       | 48     | train
5 | fc       | Linear            | 1.4 M  | train
6 | train_f1 | MulticlassF1Score | 0      | train
7 | val_f1   | MulticlassF1Score | 0      | train
-------------------------------------------------------
1.4 M     Trainable params
0         Non-trainable params
1.4 M     Total params
5.716     Total estimated mod

Epoch 49: 100%|██████████| 232/232 [00:02<00:00, 99.12it/s, v_num=10, val_loss=0.069, val_macro_f1=0.958, train_loss=0.0447, train_macro_f1=0.971]  

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 232/232 [00:02<00:00, 91.15it/s, v_num=10, val_loss=0.069, val_macro_f1=0.958, train_loss=0.0447, train_macro_f1=0.971]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

Epoch 49: 100%|██████████| 6/6 [00:00<00:00, 56.47it/s, v_num=12, val_loss=0.0191, val_macro_f1=1.000, train_loss=0.0187, train_macro_f1=1.000]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 135.27it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.02224724553525448    │
│       test_macro_f1       │            1.0            │
└───────────────────────────┴───────────────────────────┘

K7 complete
